In [1]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta
from faker import Faker
import random

# Initialize Faker
fake = Faker()

# Constants
NUM_PRODUCTS = 10
NUM_CUSTOMERS = 10
NUM_VARIANTS_PER_PRODUCT = 3
NUM_ORDERS = 100
ORDER_HISTORY_YEARS = 2

# Generate Product Data
products = []
for i in range(1, NUM_PRODUCTS + 1):
    products.append({
        'ProductID': i,
        'ProductName': fake.word(),
        'Category': fake.word(),
        'ProductDiscontinued': fake.boolean()
    })

# Generate Variant Data
variants = []
variant_id = 1
for product in products:
    num_variants = random.randint(1, NUM_VARIANTS_PER_PRODUCT)
    for _ in range(num_variants):
        variants.append({
            'VariantID': variant_id,
            'ProductID': product['ProductID'],
            'VariantName': fake.word(),
            'CurrentPrice': round(random.uniform(10.0, 100.0), 2),
            'VariantDiscontinued': fake.boolean()
        })
        variant_id += 1

# Generate PriceHistory Data
price_history = []
for variant in variants:
    start_date = datetime.now() - timedelta(days=ORDER_HISTORY_YEARS * 365)
    for _ in range(3):  # 3 price changes
        end_date = start_date + timedelta(days=random.randint(30, 180))
        price_history.append({
            'PriceID': len(price_history) + 1,
            'VariantID': variant['VariantID'],
            'Price': round(random.uniform(10.0, 100.0), 2),
            'StartDate': start_date.strftime('%Y-%m-%d'),
            'EndDate': end_date.strftime('%Y-%m-%d')
        })
        start_date = end_date

# Generate Customer Data
customers = []
for i in range(1, NUM_CUSTOMERS + 1):
    customers.append({
        'CustomerID': i,
        'CustomerName': fake.name(),
        'Email': fake.email(),
        'PhoneNumber': fake.phone_number()
    })

# Generate Address Data
addresses = []
for customer in customers:
    for _ in range(random.randint(1, 2)):
        addresses.append({
            'CustomerID': customer['CustomerID'],
            'AddressID': len(addresses) + 1,
            'AddressLine1': fake.street_address(),
            'AddressLine2': fake.secondary_address(),
            'City': fake.city(),
            'State': fake.state(),
            'PostalCode': fake.postcode(),
            'Country': fake.country(),
            'IsCurrent': fake.boolean()
        })

# Generate Order Data
orders = []
for i in range(1, NUM_ORDERS + 1):
    order_date = datetime.now() - timedelta(days=random.randint(0, ORDER_HISTORY_YEARS * 365))
    customer_id = random.choice(customers)['CustomerID']
    total_amount = round(random.uniform(20.0, 500.0), 2)
    orders.append({
        'OrderID': i,
        'OrderDate': order_date.strftime('%Y-%m-%d'),
        'TotalAmount': total_amount,
        'CustomerID': customer_id
    })

# Generate OrderItem Data
order_items = []
for order in orders:
    num_items = random.randint(1, 5)
    for _ in range(num_items):
        variant = random.choice(variants)
        quantity = random.randint(1, 10)
        order_items.append({
            'VariantID': variant['VariantID'],
            'OrderItemID': len(order_items) + 1,
            'OrderID': order['OrderID'],
            'Quantity': quantity,
            'PriceAtOrder': variant['CurrentPrice']
        })

# Convert to DataFrames
df_products = pd.DataFrame(products)
df_variants = pd.DataFrame(variants)
df_price_history = pd.DataFrame(price_history)
df_customers = pd.DataFrame(customers)
df_addresses = pd.DataFrame(addresses)
df_orders = pd.DataFrame(orders)
df_order_items = pd.DataFrame(order_items)

# Create a SQLite database and insert the data
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Create tables
cursor.execute('''
CREATE TABLE Products (
    ProductID INTEGER PRIMARY KEY,
    ProductName TEXT,
    Category TEXT,
    ProductDiscontinued BOOLEAN
)
''')

cursor.execute('''
CREATE TABLE Variants (
    VariantID INTEGER PRIMARY KEY,
    ProductID INTEGER,
    VariantName TEXT,
    CurrentPrice REAL,
    VariantDiscontinued BOOLEAN,
    FOREIGN KEY (ProductID) REFERENCES Products (ProductID)
)
''')

cursor.execute('''
CREATE TABLE PriceHistory (
    PriceID INTEGER PRIMARY KEY,
    VariantID INTEGER,
    Price REAL,
    StartDate DATE,
    EndDate DATE,
    FOREIGN KEY (VariantID) REFERENCES Variants (VariantID)
)
''')

cursor.execute('''
CREATE TABLE Customers (
    CustomerID INTEGER PRIMARY KEY,
    CustomerName TEXT,
    Email TEXT,
    PhoneNumber TEXT
)
''')

cursor.execute('''
CREATE TABLE Addresses (
    AddressID INTEGER PRIMARY KEY,
    CustomerID INTEGER,
    AddressLine1 TEXT,
    AddressLine2 TEXT,
    City TEXT,
    State TEXT,
    PostalCode TEXT,
    Country TEXT,
    IsCurrent BOOLEAN,
    FOREIGN KEY (CustomerID) REFERENCES Customers (CustomerID)
)
''')

cursor.execute('''
CREATE TABLE Orders (
    OrderID INTEGER PRIMARY KEY,
    OrderDate DATE,
    TotalAmount REAL,
    CustomerID INTEGER,
    FOREIGN KEY (CustomerID) REFERENCES Customers (CustomerID)
)
''')

cursor.execute('''
CREATE TABLE OrderItems (
    OrderItemID INTEGER PRIMARY KEY,
    OrderID INTEGER,
    VariantID INTEGER,
    Quantity INTEGER,
    PriceAtOrder REAL,
    FOREIGN KEY (OrderID) REFERENCES Orders (OrderID),
    FOREIGN KEY (VariantID) REFERENCES Variants (VariantID)
)
''')

# Insert data into tables
df_products.to_sql('Products', conn, if_exists='append', index=False)
df_variants.to_sql('Variants', conn, if_exists='append', index=False)
df_price_history.to_sql('PriceHistory', conn, if_exists='append', index=False)
df_customers.to_sql('Customers', conn, if_exists='append', index=False)
df_addresses.to_sql('Addresses', conn, if_exists='append', index=False)
df_orders.to_sql('Orders', conn, if_exists='append', index=False)
df_order_items.to_sql('OrderItems', conn, if_exists='append', index=False)

# Commit the changes
conn.commit()

# SQL Queries
# 1. Retrieve the top 5 customers who have made the highest average order amounts in the last 6 months
query1 = '''
SELECT 
    C.CustomerID, 
    C.CustomerName,
    AVG(O.TotalAmount) AS AverageOrderAmount
FROM 
    Orders O
JOIN 
    Customers C ON O.CustomerID = C.CustomerID
WHERE 
    O.OrderDate >= DATE('now', '-6 months')
GROUP BY 
    C.CustomerID, C.CustomerName
ORDER BY 
    AverageOrderAmount DESC
LIMIT 5;
'''
top_customers_last_6_months = pd.read_sql(query1, conn)
print("Top 5 customers with highest average order amounts in the last 6 months:")
print(top_customers_last_6_months)

# 2. Retrieve the list of customers whose order value is lower this year compared to the previous year
query2 = '''
SELECT 
    C.CustomerID, 
    C.CustomerName,
    COALESCE(SUM(CASE WHEN strftime('%Y', O.OrderDate) = strftime('%Y', 'now') THEN O.TotalAmount END), 0) AS CurrentYearTotal,
    COALESCE(SUM(CASE WHEN strftime('%Y', O.OrderDate) = strftime('%Y', 'now', '-1 year') THEN O.TotalAmount END), 0) AS PreviousYearTotal
FROM 
    Orders O
JOIN 
    Customers C ON O.CustomerID = C.CustomerID
GROUP BY 
    C.CustomerID, C.CustomerName
HAVING 
    CurrentYearTotal < PreviousYearTotal;
'''
customers_lower_order_value = pd.read_sql(query2, conn)
print("\nCustomers whose order value is lower this year compared to the previous year:")
print(customers_lower_order_value)

# 3. Create a table showing cumulative purchases by a particular customer, broken down by product category
query3 = '''
SELECT 
    C.CustomerID,
    C.CustomerName,
    P.Category,
    SUM(OI.Quantity * OI.PriceAtOrder) AS CumulativePurchase
FROM 
    Orders O
JOIN 
    OrderItems OI ON O.OrderID = OI.OrderID
JOIN 
    Variants V ON OI.VariantID = V.VariantID
JOIN 
    Products P ON V.ProductID = P.ProductID
JOIN 
    Customers C ON O.CustomerID = C.CustomerID
GROUP BY 
    C.CustomerID, C.CustomerName, P.Category
ORDER BY 
    C.CustomerID, P.Category;
'''
cumulative_purchases = pd.read_sql(query3, conn)
print("\nCumulative purchases by customer, broken down by product category:")
print(cumulative_purchases)

# 4. Retrieve the list of top 5 selling products, further bifurcate the sales by product variants
query4 = '''
WITH ProductSales AS (
    SELECT 
        P.ProductID, 
        P.ProductName, 
        SUM(OI.Quantity) AS TotalQuantitySold
    FROM 
        OrderItems OI
    JOIN 
        Variants V ON OI.VariantID = V.VariantID
    JOIN 
        Products P ON V.ProductID = P.ProductID
    GROUP BY 
        P.ProductID, P.ProductName
)
SELECT 
    PS.ProductID,
    PS.ProductName,
    PS.TotalQuantitySold,
    V.VariantID,
    V.VariantName,
    SUM(OI.Quantity) AS VariantQuantitySold
FROM 
    ProductSales PS
JOIN 
    Variants V ON PS.ProductID = V.ProductID
JOIN 
    OrderItems OI ON V.VariantID = OI.VariantID
GROUP BY 
    PS.ProductID, PS.ProductName, V.VariantID, V.VariantName, PS.TotalQuantitySold
ORDER BY 
    PS.TotalQuantitySold DESC, VariantQuantitySold DESC
LIMIT 5;
'''
top_selling_products = pd.read_sql(query4, conn)
print("\nTop 5 selling products, bifurcated by product variants:")
print(top_selling_products)


Top 5 customers with highest average order amounts in the last 6 months:
   CustomerID    CustomerName  AverageOrderAmount
0           7   Aaron Alvarez          391.210000
1           8     Shaun Young          385.996667
2           2  Donna Chambers          370.760000
3           6      Laura Boyd          308.225000
4          10  Kimberly Haley          255.185000

Customers whose order value is lower this year compared to the previous year:
   CustomerID            CustomerName  CurrentYearTotal  PreviousYearTotal
0           1           Michael Baker            424.11            1959.18
1           2          Donna Chambers            370.76             802.91
2           3            Calvin Davis            463.60            1889.64
3           4  Mrs. Bailey Golden DVM            746.16            1855.35
4           7           Aaron Alvarez            391.21            2106.56
5           8             Shaun Young           1157.99            1354.24
6           9          